In [10]:
import urllib.request
import re
import json

# TODO: link to docs from generated menus

files = ['blackbox', 'pathtrack', 'posdimsols', 'examfams', 'numschub', 'newtopes',
         'modphcpy2c3'] # 'dashboard', 'phcbwulf',
urls = ['http://homepages.math.uic.edu/~jan/phcpy_doc_html/_sources/{}.rst.txt'.format(f)
        for f in files]

def get(url):
    txt = urllib.request\
          .urlopen(url)\
          .read().decode('utf-8')

    re.DOTALL = True
    sec = re.findall(r'\n(.*)\n-+\n', txt)

    txts = re.split(r'\n(?:.*)\n-+\n', txt)
    txts = txts[1:] # omit intro before first ----

    # TODO: describe individual texts in a section
    
    assert(len(sec) == len(txts))
    return sec, txts

def munge(sec, txts, chap, verbose=False):
    exs = [re.findall(r'::\s*\n((?:   .*\n)*)', txt)
       for txt in txts]

    ret = []
    for i in range(len(txts)):
        if verbose: print(sec[i], len(exs[i]), sep='\n')

        res = []
        for j,ex in enumerate(exs[i]):
            if verbose: print(ex)
                
            # extract lines of input
            lines = re.findall(r'(?:>>>\s?(.*))+', ex) if '>>>' in ex \
                    else re.findall(r'(?:   (.*))+', ex)
            
            res.append("""
            {{
            'name' : "{which}",
            'snippet' : {example},
            }}""".format(which=j, example=json.dumps(lines)))
            # verify: " occuring in examples are jsonified correctly
            
        ret.append("""
        {{
        'name' : "{section}",
        'sub-menu' : [{examples}],
        }}""".format(section=sec[i], examples=','.join(res)))
        # note: explicitly flatten the list, as repr(list) encodes escapes
        
    # var {chapter} = 
    return """
    {{
    'name' : '{chapter}',
    'sub-menu' : [{sections}],
    }}"""\
        .format(chapter=chap, sections=','.join(ret))

In [11]:
print( munge(*get(urls[0]), chap="blackbox") )


    {
    'name' : 'blackbox',
    'sub-menu' : [
        {
        'name' : "solving random trinomials",
        'sub-menu' : [
            {
            'name' : "0",
            'snippet' : ["from phcpy.solver import random_trinomials", "f = random_trinomials()"],
            },
            {
            'name' : "1",
            'snippet' : ["terms = f[0].split('+(')", "for t in terms: print(t)"],
            },
            {
            'name' : "2",
            'snippet' : ["from phcpy.solver import solve", "s = solve(f, silent=True)", "len(s)", "print(s[2])"],
            }],
        },
        {
        'name' : "representations of solutions of polynomial systems ",
        'sub-menu' : [
            {
            'name' : "0",
            'snippet' : ["print(s[0])", "from phcpy.solutions import strsol2dict", "d = strsol2dict(s[0])", "d.keys()", "d['x1']"],
            },
            {
            'name' : "1",
            'snippet' : ["from phcpy.solutions import evaluate", "

In [12]:
with open("make_snippets.js", 'w') as out:
    print(
"""
require(["nbextensions/snippets_menu/main"], function (snippets_menu) {{
    snippets_menu.options['menus'] = [{{
        'name' : 'PHCpy',
        'sub-menu-direction' : 'left',
        'sub-menu' : [{}],
    }}];
}});""".format(',\n'.join([munge(*get(u), chap=f) for u,f in zip(urls, files)]))
    , file=out)